In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns

In [2]:
CSV_PATH = r"NeoJaundice\chd_jaundice_published_2.csv"
IMG_DIR = r"NeoJaundice\images"
df = pd.read_csv(CSV_PATH)
print("Dataset preview:")
print(df.head())

Dataset preview:
   patient_id   image_idx gender  gestational_age  age(day)  weight  \
0           3  0003-1.jpg      F               40       5.2    3280   
1           3  0003-2.jpg      F               40       5.2    3280   
2           3  0003-3.jpg      F               40       5.2    3280   
3          35  0035-1.jpg      M               39       8.7    3760   
4          35  0035-2.jpg      M               39       8.7    3760   

   blood(mg/dL)  Treatment  
0           3.9          0  
1           3.9          0  
2           3.9          0  
3          12.2          0  
4          12.2          0  


In [3]:
image_data = []
labels = []
for idx, row in df.iterrows():
    img_path = os.path.join(IMG_DIR, row['image_idx']) 
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256, 256)) 
        image_data.append(img)
        labels.append(row['blood(mg/dL)'])   
image_data = np.array(image_data)
labels = np.array(labels)

print(f"\nLoaded {len(image_data)} images.")


Loaded 2235 images.


In [ ]:
preprocess_image(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), size)
    img_hsv = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2HSV), size)
    img_rgb = img_rgb / 255.0
    img_hsv = img_hsv / 255.0
    return img_rgb, img_hsv

SyntaxError: invalid syntax (1369255365.py, line 1)

In [7]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
input_shape = (128, 128, 3)

# RGB branch
input_rgb = Input(shape=input_shape, name='rgb_input')
x_rgb = Conv2D(32, (3,3), activation='relu')(input_rgb)
x_rgb = MaxPooling2D((2,2))(x_rgb)
x_rgb = Conv2D(64, (3,3), activation='relu')(x_rgb)
x_rgb = MaxPooling2D((2,2))(x_rgb)
x_rgb = Flatten()(x_rgb)

# HSV branch
input_hsv = Input(shape=input_shape, name='hsv_input')
x_hsv = Conv2D(32, (3,3), activation='relu')(input_hsv)
x_hsv = MaxPooling2D((2,2))(x_hsv)
x_hsv = Conv2D(64, (3,3), activation='relu')(x_hsv)
x_hsv = MaxPooling2D((2,2))(x_hsv)
x_hsv = Flatten()(x_hsv)

# Combine features
combined = Concatenate()([x_rgb, x_hsv])
dense = Dense(64, activation='relu')(combined)
output = Dense(1, activation='linear')(dense)  # Linear for regression

model = Model(inputs=[input_rgb, input_hsv], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [11]:
model.fit(
    [rgb_images, hsv_images],
    labels,
    epochs=20,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/20


ValueError: in user code:

    File "c:\Users\Anshumaan Karna\Desktop\Neonatal Jaundice Prediction\testenv\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Anshumaan Karna\Desktop\Neonatal Jaundice Prediction\testenv\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Anshumaan Karna\Desktop\Neonatal Jaundice Prediction\testenv\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Anshumaan Karna\Desktop\Neonatal Jaundice Prediction\testenv\lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Anshumaan Karna\Desktop\Neonatal Jaundice Prediction\testenv\lib\site-packages\keras\src\utils\traceback_utils.py", line 70,